## Copyright 2019 The TensorFlow Authors.
@Adaptado por André Hochuli

---



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#**Recurrent Neural Network (RNN) para Previsão do Tempo (multivariate)**




In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

**Dataset - Dados Climáticos [[link]](https://www.bgc-jena.mpg.de/wetter/)**
* 14 características (temp, pressão, umididade, etc). 

* Granularidade: dados coletados a cada 10 minutos desde 2003

* Utilizaremos a fração de 2009 até 2016. (Processada por François Chollet [[link]](https://www.manning.com/books/deep-learning-with-python))

In [0]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [0]:
df = pd.read_csv(csv_path)

Visualização dos Dados

In [0]:
df.head()

**Objetivo**: determinar uma temperatura no futuro (*target*). Para tal, aprenderemos o padrão da variação no histórico (*history*)

* A função abaixo retorna a fração do dataset a ser utilizada sendo:
          * history_size: janela a ser observada
          * target_size: O exato momento a ser avaliado 

















In [0]:
TRAIN_SPLIT = 300000

In [0]:
tf.random.set_seed(13)

In [0]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [0]:
features = df[features_considered]
features.index = df['Date Time']
features.head()

In [0]:
features.plot(subplots=True)

**Aqui a normalização é muito importante dado que as séries tem escalas diferentes**

In [0]:
dataset = features.values
data_mean = dataset.mean(axis=0)
data_std = dataset.std(axis=0)

In [0]:
dataset = (dataset-data_mean)/data_std

### Modelo passo simples (Single step model)
Modelo prevê o valor de um ponto no futuro baseado em valores históricos.

Abaixo um esquema de janelamento (janela de tamanho definido)


In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

Neste tutorial , iremos observar os 5 dias anteriores, ou seja, 720 observações. No entanto, as amostras serão coletadas de hora em hora ao invés de 10 em 10 minutos, visto que uma mudança drastica é pouco provavel em um curto periodo de tempo. Então, 120 observações (5x24 horas) representa o histórico de 5 dias. 

Neste caso queremos prever a temperatura 12 horas a frente ou seja, 72 observações a frente (6x12)

In [0]:
past_history = 720
future_target = 72
STEP = 6 #6 observations == one hour

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Vamos verificar o formato da primeira amostra. Perceba que existem 120 observações e 3 series. 




In [0]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [0]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [0]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

*Let*'s Training.

In [0]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

#### Predict a single step future
Com o modelo treinado, vamos prever a temperatura num dado momento (12 horas a frente). 

In [0]:
def create_time_steps(length):
  time_steps = []
  for i in range(-length, 0, 1):
    time_steps.append(i)
  return time_steps
  
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'gX', 'ro']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [0]:
for x, y in val_data_single.take(10):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

### Multi-Step model
Diferentemente do single-step, aqui iremos prever um série de dados ao invés de um único pronto. 

Manteremos o protocolo de treinamento, utilizando 5 dias de observação de hora e hora, e queremos prever as proximas 12 horas a frente (12x6 = 72 pontos).

In [0]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Let's check out a sample data-point.

In [0]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [0]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()



```
# This is formatted as code
```

Vamos visualizar uma amostra dos dados

In [0]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'gX',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In [0]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Aqui, devido a complexidade do problema, criamos um layer adicional na rede LSTM para aumentar a representação do dado. Também a saída da rede (Dense Layer) deve prover 72 valores, conforme o configurado anteriormente.

In [0]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,  #Return a sequence instead of single prediction
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))  #72 instead 1

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

In [0]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [0]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

#### Predict a multi-step future


In [0]:
for x, y in val_data_multi.take(3):
   print (len(x[255]))
   multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

#Exercícios

Para o melhor entendimento, algumas questões devem ser avaliadas!

* Se treinar por mais epocas e mais instâncias, aumenta a performance da rede? 

* A rede se comporta melhor para prever 1, 3, 5 ou 10 dias?

* Se alterar o tamanho da janela de histórico, qual deve ser impacto? É melhor aumentar ou diminuir?

* Altere os hiperparâmetros do modelo (adicione mais camadas, numero de layers, etc). Procure outras arquiteturas  na internet.
